In [16]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
#from scipy import interp, arange, exp
from ccfj import GetStationPairs
import ccfj
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
import shutil
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy import stats

In [17]:
import sys
sys.path.append('../tools_F-J')
from toollib_voro import PointinPolygon
sys.path.append(r'../tools_F-J/')
from toollib_standard import mathlib

In [18]:
with open('a-project_voro.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/output_repar_03-01/'               # Harmon server
name_project

'project_voro_v1.0/output_voro_v9.3_test/'

In [19]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_voro_v1.0/output_voro_v9.3_test/


In [20]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [21]:
dir_stack = dir_project+info_basic['dir_stack']
dir_CC = dir_CC_workspace+info_basic['name_CC']
dir_ds =  dir_project + info_basic['dir_ds']
stalistname_all = info_basic['stalistname_all']

In [22]:
def Pairs(sta):
    p = []
    nsta = len(sta)
    for ii in range(nsta):
        for jj in range(ii+1,nsta):
            p.append([sta[ii],sta[jj]])
    return p
def cal_indx(pair,nsta):
    indx = int(pair[0]*(2*nsta-pair[0]-1)/2+pair[1]-pair[0]-1)
    return indx

In [23]:
def time_window_filter(t,ncfst0,r,v_min,t0,a):
    ncfst = ncfst0.copy()
    for i in range(len(ncfst)):
        tag = r[i]/v_min
        #print(t0,tag)
        t1 = t[t>-tag-t0][t[t>-tag-t0]< tag+t0]
        start = np.where(t == t1[0])[0][0]
        end = np.where(t == t1[-1])[0][0]
        ncfst[i][start:end+1]= ncfst[i][start:end+1]* np.exp(-a*np.abs((tag-np.abs(t1))))
    return ncfst

In [24]:
def singleVoro(indx0,outname,xvyv):
    global ncfs
    global f0
    global c
    global nf
    global r
    global count
    global dir_ds
    global nsta

    global t0
    global a
    global v_tag

    filename = os.path.join(dir_ds,outname+'.h5')
    if os.path.exists(filename):
        os.remove(filename)
    
    subpairs = Pairs(indx0)
    indx1 = [cal_indx(pair,nsta) for pair in subpairs]
    ncfsi = ncfs[indx1,:]
    counti = count[indx1]
    ri = r[indx1]
    indx = np.argsort(ri)
    ri = ri[indx]
    ncfsi = ncfsi[indx,:]
    counti = counti[indx]
    ncfsi = ncfsi[counti!=0]
    ri = ri[counti!=0]
    counti = counti[counti!=0]
    for i in range(len(ri)):
        ncfsi[i,:] = ncfsi[i,:]/counti[i]

    dt = 1/np.max(f0)
    t = (np.linspace(-len(f0)-1,len(f0)-1,2*(len(f0)-1))+0.5)*dt/2
    ncfst_linear = mathlib.freq_time(ncfsi)
    ncfst1 = time_window_filter(t,ncfst_linear,r,v_tag,t0,a)
    ncfs_sum_remove = mathlib.time_freq(ncfst1)

    
    ds_linear = ccfj.fj_noise(np.real(ncfsi),ri,c,f0,itype=1,func=1)
    ds_remove = ccfj.fj_noise(np.real(ncfs_sum_remove),ri,c,f0,itype=1,func=1)

    h5file=h5py.File(filename,'w')
    h5file.create_dataset('ds_linear',data=ds_linear)
    h5file.create_dataset('ds_remove',data=ds_remove)
    h5file.create_dataset('c',data=c)
    h5file.create_dataset('f',data=f0)
    h5file.create_dataset('indx',data=indx0)
    h5file.create_dataset('xvyv',data=xvyv)
    h5file.close()

In [25]:
stainfo = pd.read_excel(stalistname_all)
nsta = len(stainfo.iloc[:,0])
staloc = np.asarray([stainfo['longitude'],stainfo['latitude']]).T
# The KDE of the stations' locations
kernel = stats.gaussian_kde(staloc.T)

In [26]:
ncffile = h5py.File( dir_CC+ 'gather_all.h5','r')
ncfs = ncffile['ncfs'][:]
f0 = ncffile['f'][:]
count = ncffile['count'][:]
r = ncffile['r'][:]
ncffile.close()
nf = info_basic['nf']

In [27]:
c_min = 0.200
c_max = 2
c_num = 800
info_basic['c_min'] = c_min
info_basic['c_max'] = c_max
info_basic['c_num'] = c_num
c = np.linspace(c_min,c_max,c_num)


In [28]:
# VoroTomo Partition times
n_voro = 10
# number of Voro cells  range
kmin = 20
kmax = 40

# Set a larger range of the stations to facilitate the calculation of Voronoi cells
# the points0 are not used in the inner Voronoi cells generating
Radius = 10 
x0 = np.mean(staloc[:,0])
y0 = np.mean(staloc[:,1])
minx = min(staloc[:,0]);maxx = max(staloc[:,0])
miny = min(staloc[:,1]);maxy = max(staloc[:,1])
points0 = []
for i in range(36):
    points0.append([x0+np.cos(i/18*np.pi),y0+np.sin(i/18*np.pi)])
points0=np.asarray(points0)

In [29]:
info_basic['n_voro'] = n_voro
info_basic['kmin'] = kmin
info_basic['kmax'] = kmax
info_basic['Radius'] = Radius

In [30]:
v_tag = 2
a = 100
t0 = 0.01

In [31]:
# Starting Voronoi Partitioning
key_subworks = []
for ii in range(n_voro):
    print('start:' + str(ii))
    # random the number of Voronoi cells
    k = np.random.randint(kmin,kmax)
    # Using the KDE PDF to generate Voronoi cells
    points = kernel.resample(k).T 
    # Using the Uniform distribution to generate Voronoi cells
    '''
    points = np.random.rand(k,2)
    points[:,0] = points[:,0]*(maxx-minx) + minx
    points[:,1] = points[:,1]*(maxy-miny) + miny
    '''
    # Voronoi Partition
    points = np.concatenate((points,points0))
    vor = Voronoi(points)
    areas = []
    for j in range(k):
        xv = vor.vertices[vor.regions[vor.point_region[j]],0]
        yv = vor.vertices[vor.regions[vor.point_region[j]],1]
        _in,_on = PointinPolygon.inpolygon(staloc[:,0],staloc[:,1],xv,yv)
        indx = np.array([i for i,x in enumerate(_in) if x])
        if len(indx)>10:
            areas.append([indx,np.array([xv,yv])])
    vor.close()
    for j in range(len(areas)):
        key_subwork = 'vor'+str(ii)+'_'+str(j)
        key_subworks.append(key_subwork)
        print(ii,j,len(areas[j][0]))
        # calculate the F-J spectrum
        singleVoro(areas[j][0],key_subwork,areas[j][1])

start:0
0 0 14
0 1 43
0 2 81
0 3 39
0 4 50
0 5 22
0 6 26
0 7 34
0 8 27
0 9 59
0 10 49
0 11 25
0 12 52
0 13 31
0 14 62
0 15 25
0 16 55
0 17 57
0 18 14
0 19 12
0 20 15
0 21 25
0 22 14
0 23 20
0 24 44
0 25 26
0 26 26
0 27 37
0 28 62
0 29 52
start:1
1 0 55
1 1 109
1 2 20
1 3 13
1 4 38
1 5 43
1 6 15
1 7 143
1 8 19
1 9 44
1 10 21
1 11 28
1 12 68
1 13 32
1 14 39
1 15 43
1 16 47
1 17 29
1 18 15
1 19 43
1 20 61
1 21 82
1 22 26
1 23 28
1 24 47
start:2
2 0 48
2 1 52
2 2 82
2 3 68
2 4 20
2 5 28
2 6 26
2 7 54
2 8 22
2 9 12
2 10 16
2 11 29
2 12 25
2 13 52
2 14 28
2 15 36
2 16 29
2 17 44
2 18 74
2 19 34
2 20 54
2 21 69
2 22 148
2 23 48
start:3
3 0 20
3 1 77
3 2 34
3 3 17
3 4 25
3 5 102
3 6 27
3 7 32
3 8 50
3 9 21
3 10 72
3 11 13
3 12 23
3 13 71
3 14 18
3 15 106
3 16 59
3 17 15
3 18 28
3 19 159
3 20 35
3 21 104
start:4
4 0 41
4 1 23
4 2 18
4 3 60
4 4 17
4 5 34
4 6 59
4 7 18
4 8 16
4 9 28
4 10 43
4 11 15
4 12 14
4 13 26
4 14 36
4 15 59
4 16 45
4 17 11
4 18 18
4 19 105
4 20 12
4 21 68
4 22 59
4 23 27
4 

In [32]:
info_basic['key_subworks'] = key_subworks
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)